In [4]:
import pandas as pd
def to_dataframe(texts):
    ids = []
    labels = []
    parents = []
    info = []
    levels = []

    for idx, text in enumerate(texts):
        parts = text.split('-')
        for i, part in enumerate(parts):
            ids.append("-".join(parts[:i+1]))
            labels.append(part)
            parents.append("-".join(parts[:i]) if i > 0 else "")
            info.append(part)
            levels.append(i+1)
    return pd.DataFrame({
        "ids": ids,
        "labels": labels,
        "parents": parents,
        "info": info,
        "level": levels
    })
    
# 函数功能：将文本转换为dataframe


已经可以自定调色，自定义文字行数，取消了回调函数

In [14]:
with open('example.txt', 'r') as f:
    example = f.read()

input_list = example.split('\n')
print(input_list)
input_list = to_dataframe(input_list)
print(input_list)

['1.虚拟现实VR-虚拟现实是一种通过计算机生成的模拟环境，使用户能够与虚拟世界进行互动和沉浸式体验-虚拟现实（VR）是一种通过计算机技术创造的模拟环境，让用户能够身临其境地感受和探索虚拟世界。虚拟现实技术已经广泛应用于游戏、娱乐、教育、培训和医疗等领域。通过戴上虚拟现实头显和操纵手柄，用户可以沉浸在一个由计算机生成的虚拟环境中，并与虚拟世界进行互动。虚拟现实技术利用先进的图形处理和感知技术，为用户呈现逼真的视觉和听觉效果，从而创造出一种身临其境的感觉。在游戏领域，虚拟现实为玩家提供了全新的游戏体验，让他们可以身临其境地参与游戏世界。在教育领域，虚拟现实可以用于模拟实验、培训和虚拟旅游等方面。在医疗领域，虚拟现实可以帮助医生进行手术模拟、病例演示和康复训练等。通过不断的创新和发展，虚拟现实技术有望在未来为人们带来更加逼真和丰富的体验。', '2.区块链技术-区块链是一种分布式的账本技术，通过去中心化的方式实现信息的安全存储和交易的可追溯性-区块链技术是一种基于密码学原理的分布式账本技术，它通过去中心化的方式将交易信息记录在一个不断增长的区块链上。每个区块包含了一批交易记录，并通过哈希值链接到前一个区块，形成了一个不可篡改的链式结构。区块链的分布式特性使得信息存储在多个节点上，提高了数据的安全性和可靠性。区块链的应用领域广泛，包括金融、供应链管理、智能合约、数字资产等。在金融领域，区块链可以实现去中心化的数字货币交易，提高交易的透明度和效率。在供应链管理中，区块链可以追踪和验证商品的来源和真实性，减少欺诈和假冒产品的风险。智能合约是基于区块链的自动化合约，可以在无需第三方介入的情况下执行和验证合约条款。此外，区块链还为数字资产的创建、交易和管理提供了安全和可靠的基础设施。尽管区块链技术具有许多潜在的优势，但也面临着一些挑战，如可扩展性、隐私保护和法律法规等方面的问题。随着技术的不断发展和创新，区块链有望在未来的各个领域发挥重要作用。', '3.物联网（IoT）-物联网是将各种设备、传感器和物体连接到互联网，实现智能化和自动化的网络系统。-物联网（Internet of Things，IoT）是一种将传感器、设备和物体连接到互联网的技术。通过物联网，各种物体可以相互通信和交互，实现智能化和自动化的功能。物联网的核心是将物体与互联网连接起来，使其能够收集和共享数据。这些

In [5]:
import textwrap
def auto_wrap_text(text, max_width):
    wrapped_text = textwrap.wrap(text, max_width)
    wrapped_text = "<br>".join(wrapped_text)
    return wrapped_text

def apply_wrap(row):
    if row['level'] == 1:
        return auto_wrap_text(row['labels'], max_width=10)
    elif row['level'] == 2:
        return auto_wrap_text(row['labels'], max_width=20)
    else:
        return auto_wrap_text(row['labels'], max_width=20)


# 尝试不同的内容的换行
# 不同level的换行

In [6]:
import random
import colorsys

base_hue = random.random()  # 在Python的colorsys模块中，色调的范围是0-1，所以我们需要把角度除以360

# 生成色系相似的8个颜色
base_colors = []
for i in range(8):
    # 在色调上加上一个小的偏移（在这里是i*10度），并确保结果在0-1范围内
    hue = (base_hue + i*10/360) % 1
    # 使用饱和度1和亮度0.5生成颜色（这将生成一个鲜艳的颜色）
    color = colorsys.hls_to_rgb(hue, 0.5, 1)
    # 将颜色从RGB空间转换到十六进制颜色代码
    base_colors.append("#"+''.join([f"{int(val*255):02x}" for val in color]))
    

# 转换十六进制颜色代码为RGB颜色值（范围是0-1）
base_colors_rgb = [[int(color[i:i+2], 16) / 255 for i in range(1, 6, 2)] for color in base_colors]

# 节点和层级的关系，假设我们有8个段落，每个段落有3个层级
nodes = ["P{}-L{}".format(p, l) for p in range(8) for l in range(3)]
levels = [l for p in range(8) for l in range(3)]

# 根据节点的层级和对应的基础颜色来生成颜色
node_colors = []
for i in range(8):
    for j in range(3):
        # 随着层级增加，将颜色的亮度增加
        h, l, s = colorsys.rgb_to_hls(*base_colors_rgb[i])
        color = colorsys.hls_to_rgb(h, min(1, l + 0.2*j), s)  # 每次增加20%的亮度，最大为1
        node_colors.append("rgba"+str(tuple([int(255*val) for val in color] + [1.0])) + ")")


In [8]:
df

,ids,labels,parents,info,level
0,. **人工智能的历史与理论基础**,. **人工智能的历<br>史与理论基础**,,info about . **人工智能的历史与理论基础**,1
1,. **人工智能的历史与理论基础**/早期的人工智能研究与应用,早期的人工智能研究与应用,/. **人工智能的历史与理论基础**,info about 早期的人工智能研究与应用,2
2,. **人工智能的历史与理论基础**/机器人初期研究,机器人初期研究,/. **人工智能的历史与理论基础**,info about 机器人初期研究,2
3,. **人工智能的历史与理论基础**/Eliza和Shakey,Eliza和Shakey,/. **人工智能的历史与理论基础**,info about Eliza和Shakey,2
4,. **人工智能的历史与理论基础**/Eliza的对话系统原理,Eliza的对话系统原理,/. **人工智能的历史与理论基础**,info about Eliza的对话系统原理,2
...,...,...,...,...,...
130,. **人工智能的未来发展**/保护数据隐私,保护数据隐私,/. **人工智能的未来发展**,info about 保护数据隐私,2
131,. **人工智能的未来发展**/AI的伦理问题,AI的伦理问题,/. **人工智能的未来发展**,info about AI的伦理问题,2
132,. **人工智能的未来发展**/人工智能的影响：工作、社会、经济等,人工智能的影响：工作、社会、经济等,/. **人工智能的未来发展**,info about 人工智能的影响：工作、社会、经济等,2
133,. **人工智能的未来发展**/AI对工作的影响,AI对工作的影响,/. **人工智能的未来发展**,info about AI对工作的影响,2


In [7]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import json
import textwrap
import numpy as np
import plotly.colors as colors
import matplotlib.pyplot as plt
import random
import colorsys

# 定义颜色刻度，根据 level 的最小值和最大值生成渐变色
color_scale = colors.sequential.Blues

# 生成模拟数据


# df = pd.DataFrame(input_list)

# df["labels"] = df["labels"].apply(lambda x: auto_wrap_text(x, max_width=20))  # 20 是你想要的最大行长度
df["labels"] = df.apply(apply_wrap, axis=1)





fig = go.Figure(go.Treemap(
    ids=df['ids'],
    labels=df['labels'],
    parents=df['parents'],
    # values=df['level'],
    # values=[10]*len(df['level']),
    maxdepth=2,
    # text=df['labels'],
    insidetextfont=dict(size=30, color='#FFFFFF'),
    outsidetextfont=dict(size=15, color='#FF0000', family='Arial'),
    # hoverlabel=dict(
    #     font=dict(
    #         color='#000000',
    #         size=8,
    #         family='Arial'
    #     ),
    #     bgcolor='lightyellow'
    # ),
    # pathbar_textfont_size=15,
    # textfont=dict(size=20, exponent=-1) 
    
    # marker_colorscale = 'Blues',  # 使用 'Blues' 颜色方案
    # color_continuous_scale='Twilight',
    marker_colors=node_colors,  # 使用标签长度作为权重
    
    
    # tiling = dict(packing='slice-dice')  # 使用 'slice-dice' 布局算法
    tiling=dict(
    packing='squarify',  # Set packing algorithm to 'squarify'
    pad=0.2
    )
))

fig.update_layout(
    # title="人工智能在不同行业的应用",  # 添加标题
    title={
    'text': "<b>人工智能在不同行业的应用",
    'x': 0.5,  # 将标题的 x 位置设置为 0.5，即图形的中心
    'xanchor': 'center',  # 将标题的 x 锚点设置为 'center'，即标题的中心
    # 'y' : 0.96 , # 高度调整
    },
    title_font=dict(
    family="Courier New, monospace",  # 设置字体
    size=20,  # 设置字体大小
    color="RebeccaPurple"  # 设置字体颜色
    ),
    # uniformtext=dict(minsize=15, mode='hide'), 
    margin = dict(t=50, l=25, r=25, b=25),
    width=600,
    height=600,
    # plot_bgcolor="#0000FF",  # 设置画布的背景颜色
    # autosize=True,
)

# fig.update_layout(
#     # 
# )

fig.update_traces(marker=dict(cornerradius=20)) # 圆角
fig.show()


# app = JupyterDash(__name__)
# 召回函数
# app.layout = html.Div([
#     dcc.Graph(
#         id='my-graph',
#         figure=fig
#     ),
#     # html.Pre(id='click-data', style={'padding': '10px'}),
#     # html.Div(id='text-storage', style={'display': 'none'}),
#     # html.Div(id='node-clicked', style={'display': 'none'})
# ])

# @app.callback(
#     Output('my-graph', 'figure'),
#     Output('text-storage', 'children'),
#     Output('node-clicked', 'children'),
#     Input('my-graph', 'clickData'),
#     State('my-graph', 'figure'),
#     State('text-storage', 'children'),
#     State('node-clicked', 'children')
# )

# def update_figure(clickData, fig, stored_text, clicked_node):
#     if clickData:
#         point_number = clickData['points'][0]['pointNumber']
#         if stored_text is None:
#             stored_text = json.dumps(fig['data'][0]['text'])
#         if clicked_node is None or clicked_node != point_number:
#             fig['data'][0]['text'] = json.loads(stored_text)
#             fig['data'][0]['insidetextfont']['size'] = 30  # 将字体大小设置为30
#             clicked_node = point_number
#         else:
#             fig['data'][0]['text'] = json.loads(stored_text)
#             fig['data'][0]['insidetextfont']['size'] = 20  # 将字体大小重设为20
#             clicked_node = None
#     else:
#         if stored_text is not None:
#             fig['data'][0]['text'] = json.loads(stored_text)
#         fig['data'][0]['insidetextfont']['size'] = 20  # 将字体大小重设为20
#         clicked_node = None

#     return fig, stored_text, clicked_node


# app.run_server(mode='inline', port=8054)


In [7]:
len(node_colors)

24

In [164]:
df = pd.DataFrame(input_list)
df

,ids,labels,parents,info,level
0,1.虚拟现实VR,1.虚拟现实VR,,1.虚拟现实VR,1
1,1.虚拟现实VR-虚拟现实是一种通过计算机生成的模拟环境，使用户能够与虚拟世界进行互动和沉浸式体验,虚拟现实是一种通过计算机生成的模拟环境，使用户能够与虚拟世界进行互动和沉浸式体验,1.虚拟现实VR,虚拟现实是一种通过计算机生成的模拟环境，使用户能够与虚拟世界进行互动和沉浸式体验,2
2,1.虚拟现实VR-虚拟现实是一种通过计算机生成的模拟环境，使用户能够与虚拟世界进行互动和沉浸...,虚拟现实（VR）是一种通过计算机技术创造的模拟环境，让用户能够身临其境地感受和探索虚拟世界。...,1.虚拟现实VR-虚拟现实是一种通过计算机生成的模拟环境，使用户能够与虚拟世界进行互动和沉浸式体验,虚拟现实（VR）是一种通过计算机技术创造的模拟环境，让用户能够身临其境地感受和探索虚拟世界。...,3
3,2.区块链技术,2.区块链技术,,2.区块链技术,1
4,2.区块链技术-区块链是一种分布式的账本技术，通过去中心化的方式实现信息的安全存储和交易的可追溯性,区块链是一种分布式的账本技术，通过去中心化的方式实现信息的安全存储和交易的可追溯性,2.区块链技术,区块链是一种分布式的账本技术，通过去中心化的方式实现信息的安全存储和交易的可追溯性,2
5,2.区块链技术-区块链是一种分布式的账本技术，通过去中心化的方式实现信息的安全存储和交易的可...,区块链技术是一种基于密码学原理的分布式账本技术，它通过去中心化的方式将交易信息记录在一个不断...,2.区块链技术-区块链是一种分布式的账本技术，通过去中心化的方式实现信息的安全存储和交易的可追溯性,区块链技术是一种基于密码学原理的分布式账本技术，它通过去中心化的方式将交易信息记录在一个不断...,3
6,3.物联网（IoT）,3.物联网（IoT）,,3.物联网（IoT）,1
7,3.物联网（IoT）-物联网是将各种设备、传感器和物体连接到互联网，实现智能化和自动化的网络系统。,物联网是将各种设备、传感器和物体连接到互联网，实现智能化和自动化的网络系统。,3.物联网（IoT）,物联网是将各种设备、传感器和物体连接到互联网，实现智能化和自动化的网络系统。,2
8,3.物联网（IoT）-物联网是将各种设备、传感器和物体连接到互联网，实现智能化和自动化的网络...,物联网（Internet of Things，IoT）是一种将传感器、设备和物体连接到互联网...,3.物联网（IoT）-物联网是将各种设备、传感器和物体连接到互联网，实现智能化和自动化的网络系统。,物联网（Internet of Things，IoT）是一种将传感器、设备和物体连接到互联网...,3
9,4.自动驾驶,4.自动驾驶,,4.自动驾驶,1


In [1]:
import pandas as pd

def markdown_to_df(md_lines):
    # 初始化存储ids、labels、parents和levels的空列表
    ids = []
    labels = []
    parents = []
    levels = []
    info = []
    # 初始化父级列表，存储每一级的父级标题
    parent_list = [""] * 10

    for line in md_lines:
        # 这里假设输入是一个行的列表，而不是一个文件
        stripped_line = line.strip()
        if stripped_line.startswith("*"):
            level = 1
        else:
            level = stripped_line.count("-", 0, len(stripped_line) - len(stripped_line.lstrip("-"))) + 1
        label = stripped_line[level:].strip()
        parent = parent_list[level-1]
        ids.append(label if parent=="" else parent + '/' + label)
        labels.append(label)
        parents.append("" if parent=="" else parent if level==1 else parent_list[level-2]+'/'+parent)
        levels.append(level)
        parent_list[level] = label
        # 给info列提供一些默认值，你可以替换为实际的数据
        info.append("info about " + label)

    df = pd.DataFrame({
        "ids": ids,
        "labels": labels,
        "parents": parents,
        "info": info,
        "level": levels
    })

    return df



In [22]:
import pandas as pd

def markdown_to_df(md_lines):
    # 初始化存储ids、labels、parents和levels的空列表
    ids = []
    labels = []
    parents = []
    levels = []
    info = []
    # 初始化父级列表，存储每一级的父级标题
    parent_list = [""] * 10

    for line in md_lines:
        # 这里假设输入是一个行的列表，而不是一个文件
        stripped_line = line.lstrip(" ")
        if stripped_line.startswith("1. **"):
            level = 1
        else:
            level = line.count(" ", 0, len(line) - len(stripped_line)) // 2 + 1
        label = stripped_line[level:].strip().strip('*')
        parent = parent_list[level-1]
        ids.append(label if parent=="" else parent + '/' + label)
        labels.append(label)
        parents.append("" if parent=="" else parent if level==1 else parent_list[level-2]+'/'+parent)
        levels.append(level)
        parent_list[level] = label
        # 给info列提供一些默认值，你可以替换为实际的数据
        info.append(label)

    df = pd.DataFrame({
        "ids": ids,
        "labels": labels,
        "parents": parents,
        "info": info,
        "level": levels
    })

    return df





In [23]:
def read_markdown_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        # 使用列表推导式去掉每行的换行符
        return [line.rstrip('\n') for line in file.readlines()]

md_lines = read_markdown_file('test1.md')

df = markdown_to_df(md_lines)
print(df)


                                 ids             labels  \
0                   . **人工智能的历史与理论基础   . **人工智能的历史与理论基础   
1      . **人工智能的历史与理论基础/早期的人工智能研究与应用       早期的人工智能研究与应用   
2                早期的人工智能研究与应用/器人初期研究             器人初期研究   
3                  器人初期研究/iza和Shakey         iza和Shakey   
4               iza和Shakey/za的对话系统原理          za的对话系统原理   
..                               ...                ...   
130                  数据隐私和伦理考量/护数据隐私              护数据隐私   
131                 数据隐私和伦理考量/I的伦理问题             I的伦理问题   
132  . **人工智能的未来发展/人工智能的影响：工作、社会、经济等  人工智能的影响：工作、社会、经济等   
133        人工智能的影响：工作、社会、经济等/I对工作的影响            I对工作的影响   
134        人工智能的影响：工作、社会、经济等/I对社会的影响            I对社会的影响   

                             parents               info  level  
0                                      . **人工智能的历史与理论基础      1  
1                  /. **人工智能的历史与理论基础       早期的人工智能研究与应用      2  
2      . **人工智能的历史与理论基础/早期的人工智能研究与应用             器人初期研究      3  
3                早期的人工智能研究与应用/器

In [24]:
df.head(5)


,ids,labels,parents,info,level
0,. **人工智能的历史与理论基础,. **人工智能的历史与理论基础,,. **人工智能的历史与理论基础,1
1,. **人工智能的历史与理论基础/早期的人工智能研究与应用,早期的人工智能研究与应用,/. **人工智能的历史与理论基础,早期的人工智能研究与应用,2
2,早期的人工智能研究与应用/器人初期研究,器人初期研究,. **人工智能的历史与理论基础/早期的人工智能研究与应用,器人初期研究,3
3,器人初期研究/iza和Shakey,iza和Shakey,早期的人工智能研究与应用/器人初期研究,iza和Shakey,4
4,iza和Shakey/za的对话系统原理,za的对话系统原理,器人初期研究/iza和Shakey,za的对话系统原理,5


In [16]:
md_lines

['1. **人工智能的历史与理论基础**',
 '   - 早期的人工智能研究与应用',
 '     - 机器人初期研究',
 '       - Eliza和Shakey',
 '         - Eliza的对话系统原理',
 '         - Shakey的决策与路径规划',
 '       - 专家系统的崛起',
 '     - AI冬天的来临',
 '     - 人工智能的复兴',
 '   - AI的理论基础',
 '     - 符号主义',
 '     - 连接主义',
 '     - 进化主义',
 '   - Turing Test',
 '   - 重要的人工智能发展里程碑',
 '   - 人工智能的伦理和哲学问题',
 '',
 '2. **机器学习**',
 '   - 监督学习',
 '     - 线性回归',
 '       - 参数估计',
 '       - 损失函数',
 '       - 优化算法',
 '     - 决策树',
 '     - 神经网络',
 '   - 无监督学习',
 '     - 聚类',
 '     - 主成分分析',
 '   - 强化学习',
 '     - Q-learning',
 '     - 深度强化学习',
 '   - 深度学习',
 '     - 卷积神经网络',
 '     - 循环神经网络',
 '     - 自编码器和生成对抗网络',
 '   - 机器学习的实际应用和案例',
 '',
 '3. **自然语言处理（NLP）**',
 '   - 语言模型',
 '     - n-gram模型',
 '     - 神经网络语言模型',
 '   - 信息提取',
 '     - 实体识别',
 '     - 关系抽取',
 '   - 语义理解',
 '     - 依存解析',
 '     - 语义角色标注',
 '   - 机器翻译',
 '     - 统计机器翻译',
 '     - 神经网络机器翻译',
 '   - 文本生成',
 '     - 条件生成',
 '     - 无条件生成',
 '',
 '4. **计算机视觉**',
 '   - 图像分类',
 '     - 二分类问题',
 ' 